In [1]:
import pandas as pd
import numpy as np
import zipfile
import matplotlib.pyplot as plt
from IPython.display import clear_output, display



In [2]:
#load data
zf = zipfile.ZipFile('../content/anonymisedData.zip') 
student_vle = pd.read_csv(zf.open('studentVle.csv'))
assessments = pd.read_csv(zf.open('studentAssessment.csv'), skiprows=[128223,64073])
assessments_info = pd.read_csv(zf.open('assessments.csv'))
student_info =  pd.read_csv(zf.open('studentInfo.csv'),
                           usecols = ['code_module','code_presentation','id_student',
                                     'final_result'])
student_unregistration = pd.read_csv(zf.open('studentRegistration.csv'),
                              usecols = ['code_module','code_presentation','id_student',
                                         'date_unregistration'])

student_unregistration.head()

,code_module,code_presentation,id_student,date_unregistration
0,AAA,2013J,11391,NaN
1,AAA,2013J,28400,NaN
2,AAA,2013J,30268,12.0
3,AAA,2013J,31604,NaN
4,AAA,2013J,32885,NaN


In [3]:
assessments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173910 entries, 0 to 173909
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   id_assessment   173910 non-null  int64  
 1   id_student      173910 non-null  int64  
 2   date_submitted  173910 non-null  int64  
 3   is_banked       173910 non-null  int64  
 4   score           173737 non-null  float64
dtypes: float64(1), int64(4)
memory usage: 6.6 MB


In [4]:
assessments.drop_duplicates().info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 173910 entries, 0 to 173909
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   id_assessment   173910 non-null  int64  
 1   id_student      173910 non-null  int64  
 2   date_submitted  173910 non-null  int64  
 3   is_banked       173910 non-null  int64  
 4   score           173737 non-null  float64
dtypes: float64(1), int64(4)
memory usage: 8.0 MB


In [5]:
#combine module, presentation, and student id columns into one registration column
student_vle['registration'] = student_vle['code_module'] \
                                + student_vle['code_presentation'] \
                                + student_vle['id_student'].astype(str)

student_info['registration'] = student_info['code_module'] \
                            + student_info['code_presentation'] \
                            + student_info['id_student'].astype('str')

student_unregistration['registration'] = student_unregistration['code_module'] \
                            + student_unregistration['code_presentation'] \
                            + student_unregistration['id_student'].astype('str')

student_unregistration = student_unregistration[['registration','date_unregistration']]

# extract target variable, final_result
results = student_info[['registration','final_result']]

#group by registration and day
vle_group = student_vle.groupby(by = ['registration', 'date'])

#sum activities and clicks per day. activities are '.count()' because each row is an activity.
sum_activities = vle_group.count().reset_index()[['registration','date','id_site']]
sum_clicks = vle_group.sum().reset_index()[['registration','date','sum_click']]

df = pd.merge(sum_activities, sum_clicks, on=['registration','date'], how='inner',
             validate='1:1')
df = df.rename(columns = {'id_site':'sum_activities'})

df['activities_x_clicks'] = df['sum_activities'] * df['sum_click']
df = df.sort_values(by=['registration','date'])

#find statistics per student to identify outliers
grouped_by_student = student_vle.groupby('registration')
student_clicks = grouped_by_student.sum().reset_index()[['registration','sum_click']]
student_activities = grouped_by_student.count().reset_index()[['registration','id_site']]
student_stats = pd.merge(student_clicks, student_activities, on='registration')

#change this to be algorithmic rather than constant
no_outliers = student_stats[(student_stats['sum_click'] < 4035) 
                          & (student_stats['id_site'] < 1135)]

#drop outliers from dataframe, including coursework completed after course end.
df = df[df['registration'].isin(no_outliers['registration'])]
df = df[df.date <= 269]

#A little more cleanup
df = df.fillna(0)
df = df.drop_duplicates(keep='first')
df = df.rename({'score':'assessment_score'})

student_unregistration['date_unregistration'] = student_unregistration['date_unregistration'].fillna(df.date.max()+1)

df.head()

,registration,date,sum_activities,sum_click,activities_x_clicks
0,AAA2013J100893,-9,6,14,84
1,AAA2013J100893,-2,5,13,65
2,AAA2013J100893,3,1,2,2
3,AAA2013J100893,4,8,9,72
4,AAA2013J100893,5,7,21,147


In [6]:
date_range = range(df.date.min(),df.date.max()+1)
cols = ['registration']
cols.extend(date_range)


activities_clicks_and_product = pd.DataFrame()
activities_clicks_and_product['registration'] = df.registration.unique()
counter = len(date_range)

for date in date_range:
    single_date_df = df[df.date == date][['registration','sum_activities','sum_click','activities_x_clicks']]

    single_date_df.columns = ['registration'] + [f'{x}_{date}' for x in single_date_df.columns[1:]]

    activities_clicks_and_product = activities_clicks_and_product.merge(single_date_df, 
                                            how='left', 
                                            on='registration',
                                            validate = '1:m')
    
    print('remaining dates: ', counter)
    clear_output(wait=True)
    counter -= 1

activities_clicks_and_product = activities_clicks_and_product.fillna(0)

activities_clicks_and_product = activities_clicks_and_product.merge(results, 
                                                                      how='left', 
                                                                      on='registration')

activities_clicks_and_product = activities_clicks_and_product.merge(student_unregistration, 
                                                                      how='inner', 
                                                                      on='registration')

activities_clicks_and_product.to_csv('../content/activities_clicks_and_product.csv', index=False)

remaining dates:  1


# Different modules have different numbers of assessments, see below.

Ideas:
1. Add 'code_module' and/or 'code_presentation' to df
2. 

In [7]:
prediction_window = 150


full_assess = assessments.merge(assessments_info, on='id_assessment', how='left')
full_assess = full_assess[full_assess['date'] <= prediction_window]

full_assess = full_assess.dropna(axis=0, subset=['score'])
full_assess['date'] = full_assess['date'].fillna(full_assess['date_submitted'])

full_assess['registration'] = full_assess['code_module'] \
                            + full_assess['code_presentation'] \
                            + full_assess['id_student'].astype(int).astype(str)
full_assess['relative_date'] = full_assess['date_submitted'] - full_assess['date']

max_assess_count = full_assess.groupby('registration').count().max().max()
temp_assess = full_assess.sort_values(by=['date','date_submitted'])
registered = full_assess.registration.unique()
assess_timeseries = pd.DataFrame(registered, columns=['registration'])

for assess_num in range(max_assess_count):
    single_assess = temp_assess.groupby('registration').head(1)
    single_assess = single_assess[['registration','relative_date','score']]
    single_assess = single_assess.rename(columns = {'relative_date':f'relative_date_{assess_num}',
                                                          'score':f'score_{assess_num}'})
    assess_timeseries = assess_timeseries.merge(single_assess, on='registration', how='left')
    temp_assess = temp_assess.drop(index = single_assess.index)
    
assess_timeseries

,registration,relative_date_0,score_0,relative_date_1,score_1,relative_date_2,score_2,relative_date_3,score_3,relative_date_4,score_4,relative_date_5,score_5,relative_date_6,score_6,relative_date_7,score_7,relative_date_8,score_8
0,AAA2013J11391,-1.0,78.0,-1.0,85.0,-2.0,80.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AAA2013J28400,3.0,70.0,-2.0,68.0,4.0,70.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AAA2013J31604,-2.0,72.0,-3.0,71.0,-2.0,74.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AAA2013J32885,7.0,69.0,21.0,30.0,7.0,63.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AAA2013J38053,0.0,79.0,10.0,69.0,0.0,74.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25771,GGG2014J681532,14.0,60.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25772,GGG2014J682100,-3.0,66.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25773,GGG2014J687523,25.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25774,GGG2014J697314,-2.0,70.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
full_assess[(full_assess['code_module'] == 'GGG')
           & (full_assess['code_presentation'] == '2013J')].groupby('registration').count()

,id_assessment,id_student,date_submitted,is_banked,score,code_module,code_presentation,assessment_type,date,weight,relative_date
registration,,,,,,,,,,,
GGG2013J121056,1,1,1,1,1,1,1,1,1,1,1
GGG2013J122459,2,2,2,2,2,2,2,2,2,2,2
GGG2013J122636,1,1,1,1,1,1,1,1,1,1,1
GGG2013J122846,2,2,2,2,2,2,2,2,2,2,2
GGG2013J122928,2,2,2,2,2,2,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...
GGG2013J67182,2,2,2,2,2,2,2,2,2,2,2
GGG2013J80448,1,1,1,1,1,1,1,1,1,1,1
GGG2013J82341,2,2,2,2,2,2,2,2,2,2,2


In [9]:
full_assess[(full_assess['code_module'] == 'BBB')
           & (full_assess['code_presentation'] == '2013J')].groupby('registration').count()

,id_assessment,id_student,date_submitted,is_banked,score,code_module,code_presentation,assessment_type,date,weight,relative_date
registration,,,,,,,,,,,
BBB2013J100282,1,1,1,1,1,1,1,1,1,1,1
BBB2013J105527,7,7,7,7,7,7,7,7,7,7,7
BBB2013J105851,7,7,7,7,7,7,7,7,7,7,7
BBB2013J105883,7,7,7,7,7,7,7,7,7,7,7
BBB2013J106588,7,7,7,7,7,7,7,7,7,7,7
...,...,...,...,...,...,...,...,...,...,...,...
BBB2013J90243,7,7,7,7,7,7,7,7,7,7,7
BBB2013J96475,7,7,7,7,7,7,7,7,7,7,7
BBB2013J97567,7,7,7,7,7,7,7,7,7,7,7
